# 생성한 파생지수 병합

In [45]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

In [46]:
plt.rcParams["font.family"] = 'AppleGothic' if sys.platform == 'darwin' else 'Malgun Gothic'
plt.rcParams["font.size"] = 12
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.unicode_minus"] = False

## 데이터 불러오기

In [47]:
origin=read_excel('행정동병합테스트22.xlsx')
origin2=read_excel('여가생활지수1.xlsx')
origin3=read_excel('사회적약자지수1.xlsx')
origin4=read_excel('접근성지수1.xlsx')
origin5=read_excel('그린지수1.xlsx')
origin.head(1)

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3


In [48]:
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


## 통합할 데이터 추출

### `읍면동명`,`여가생활지수minmax`

In [49]:
origin2_2=origin2.filter(['읍면동명','여가생활지수minmax'])
origin2_2

,읍면동명,여가생활지수minmax
0,청운효자동,0.038298
1,사직동,0.093504
2,삼청동,0.077132
3,부암동,0.024572
4,평창동,0.007649
...,...,...
421,성내제2동,0.052895
422,성내제3동,0.042945
423,길동,0.033285
424,둔촌제1동,0.000000


### `읍면동명`,`사회적약자_지원_인프라_밀집도minmax`

In [50]:
origin3_3=origin3.filter(['읍면동명','사회적약자_지원_인프라_밀집도minmax'])
origin3_3

,읍면동명,사회적약자_지원_인프라_밀집도minmax
0,청운효자동,0.120179
1,사직동,0.033721
2,삼청동,0.078692
3,부암동,0.104086
4,평창동,0.489879
...,...,...
421,성내제2동,0.019826
422,성내제3동,0.026658
423,길동,0.058455
424,둔촌제1동,0.541383


### `읍면동명`,`접근성지수minmax`

In [51]:
origin4_4=origin4.filter(['읍면동명','접근성지수minmax'])
origin4_4

,읍면동명,접근성지수minmax
0,청운효자동,0.035322
1,사직동,0.131414
2,삼청동,0.040614
3,부암동,0.043385
4,평창동,0.018995
...,...,...
421,성내제2동,0.362445
422,성내제3동,0.239798
423,길동,0.123873
424,둔촌제1동,0.063369


### `읍면동명`,`그린지수minmax`

In [52]:
origin5_5=origin5.filter(['읍면동명','그린지수minmax'])
origin5_5

,읍면동명,그린지수minmax
0,청운효자동,0.187151
1,사직동,0.033704
2,삼청동,0.530328
3,부암동,0.530328
4,평창동,0.530960
...,...,...
421,성내제2동,0.212231
422,성내제3동,0.477489
423,길동,0.002279
424,둔촌제1동,0.371230


## 데이터 통합

### `읍면동명`,`여가생활지수minmax`

In [53]:
tmp=merge(origin,origin2_2,how='left')
tmp.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,여가생활지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,0.038298
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,0.093504
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,0.077132
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,0.024572
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,0.007649


### `읍면동명`,`사회적약자_지원_인프라_밀집도minmax`

In [54]:
tmp1=merge(tmp,origin3_3,how='left')
tmp1

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,437,337.55,4,0.352898,0,32,0,3,0.038298,0.120179
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,267,60.79,0,0.063554,0,27,0,12,0.093504,0.033721
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,110,956.51,3,1.000000,0,16,0,5,0.077132,0.078692
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,311,0.00,3,0.000000,10,34,0,5,0.024572,0.104086
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,491,1.14,3,0.001192,10,54,0,5,0.007649,0.489879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,1075,0.18,9,0.000188,4,18,1,5,0.052895,0.019826
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,913,0.35,11,0.000366,9,15,1,6,0.042945,0.026658
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,1988,4.11,17,0.004297,0,36,2,7,0.033285,0.058455
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,3,0.00,0,0.000000,7,9,0,4,0.000000,0.541383


### `읍면동명`,`접근성지수minmax`

In [55]:
tmp2=merge(tmp1,origin4_4,how='left')
tmp2.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax,접근성지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,337.55,4,0.352898,0,32,0,3,0.038298,0.120179,0.035322
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,60.79,0,0.063554,0,27,0,12,0.093504,0.033721,0.131414
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,956.51,3,1.000000,0,16,0,5,0.077132,0.078692,0.040614
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.00,3,0.000000,10,34,0,5,0.024572,0.104086,0.043385
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1.14,3,0.001192,10,54,0,5,0.007649,0.489879,0.018995


### `읍면동명`,`그린지수minmax`

In [56]:
tmp3=merge(tmp2,origin5_5,how='left')
tmp3.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,여가생활지수minmax,사회적약자_지원_인프라_밀집도minmax,접근성지수minmax,그린지수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,4,0.352898,0,32,0,3,0.038298,0.120179,0.035322,0.187151
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,0.063554,0,27,0,12,0.093504,0.033721,0.131414,0.033704
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,3,1.000000,0,16,0,5,0.077132,0.078692,0.040614,0.530328
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,3,0.000000,10,34,0,5,0.024572,0.104086,0.043385,0.530328
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3,0.001192,10,54,0,5,0.007649,0.489879,0.018995,0.530960


## 통합 데이터 확인

In [57]:
tmp3.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '여가생활지수minmax', '사회적약자_지원_인프라_밀집도minmax', '접근성지수minmax',
       '그린지수minmax'],
      dtype='object')

## 데이터 저장

In [58]:
tmp3.to_excel('행정동병합최종.xlsx',index=False)